In [2]:
from sklearn.datasets import load_diabetes
import pickle
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import os
import torch


from synthcity.metrics.eval_performance import (
    PerformanceEvaluatorMLP,
    PerformanceEvaluatorXGB,
)
from synthcity.utils import reproducibility
from synthcity.plugins import Plugins
import synthcity.logger as log
from synthcity.plugins.core.dataloader import GenericDataLoader
from DGE_utils import metric_different_datasets, mean_across_pandas, add_std, get_folder_names

reproducibility.clear_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


assert device.type == 'cuda'
Plugins(categories=["generic"]).list()


/home/bv292/miniconda/envs/gen/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.

<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.

<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.

<stdin>:1:10: fatal error: cuda.h: No such file or directory
compilation terminated.



['sdv_tvae',
 'ctgan',
 'rtvae',
 'adsgan',
 'copulagan',
 'sdv_ctgan',
 'gaussian_copula',
 'bayesian_network',
 'privbayes',
 'nflow',
 'pategan',
 'tvae',
 'dpgan']

In [3]:
from DGE_data import get_real_and_synthetic

# let's restrict ourselves to classification datasets
datasets = ['moons', 'circles', 'breast_cancer',
            'adult',  'seer', 'covid']
# ['moons', 'circles','cal_housing', 'adult', 'diabetes', 'breast_cancer',  'seer', 'cutract' ]
model_name = 'ctgan'  # synthetic data model

p_train = 0.8  # proportion of training data for generative model. Default values if None
n_models = 20  # number of models in ensemble, for each run
max_n = 2000  # maximum number of data points to use for training generative model.
nsyn = 2000  # number of synthetic data points per synthetic dataset. Defaults to same as generative training size if None

num_runs = 10  # Number of runs. Don't choose to large, since total number of synthetic datasets is num_runs*n_models

load = True  # results
load_syn = True  # data
save = True  # save results and data

outlier = False

verbose = False

if nsyn is None:
    nsyn = max_n


In [4]:
from DGE_utils import get_folder_names, tt_predict_performance, cat_dl
from DGE_data import get_real_and_synthetic
from DGE_experiments import cross_val
import pandas as pd
from sklearn.model_selection import KFold


In [8]:
num_runs = 5
model_type = 'largest_mlp'
model_name = 'ctgan_deep'
nsyn = 5000
max_n = 5000
p_train = 0.8
n_models = 20
cross_fold = 5
load_syn = True
load = True
save = True
verbose = True

scores_s_all = {}
scores_r_all = {}


for dataset in ['seer']:
    workspace_folder, results_folder = get_folder_names(
        dataset, model_name, max_n=max_n, nsyn=nsyn)

    X_gt, X_syns = get_real_and_synthetic(dataset=dataset,
                                          p_train=p_train,
                                          n_models=n_models*num_runs,
                                          model_name=model_name,
                                          load_syn=load_syn,
                                          verbose=verbose,
                                          max_n=max_n,
                                          nsyn=nsyn)

    print(f'Dataset {dataset}\n')

    scores_s, scores_r = cross_val(X_gt,
                                  X_syns,
                                  workspace_folder=workspace_folder,
                                  results_folder=results_folder,
                                  save=save,
                                  load=load,
                                  task_type=model_type,
                                  cross_fold=cross_fold,
                                  verbose=verbose
                                  )

    scores_s_all[dataset] = scores_s
    scores_r_all[dataset] = scores_r


n_total 20000 n_train: 5000
Dataset seer

Computing means and stds
Oracle
Run 0 approach Oracle split 0
Run 0 approach Oracle split 1
Run 0 approach Oracle split 2
Run 0 approach Oracle split 3
Run 0 approach Oracle split 4
Naive
Run 0 approach Naive split 0
Run 0 approach Naive split 1
Run 0 approach Naive split 2
Run 0 approach Naive split 3
Run 0 approach Naive split 4
DGE$_{20]$
Run 0 approach DGE$_{20]$ split 0
Run 0 approach DGE$_{20]$ split 1


In [ ]:
for dataset in ['seer']:
    print(dataset)
    print(scores_s_all[dataset].groupby('approach').mean())
    print(scores_r_all[dataset].groupby('approach').mean())

seer
                          AUC       Acc        F1  Precision    Recall  \
approach                                                                 
DGE$_{20]$           0.859100  0.780512  0.718981   0.742539  0.716457   
DGE$_{20}$ (concat)  0.858001  0.782374  0.720023   0.747587  0.712358   
Naive                0.838294  0.911840  0.568821   0.596912  0.553865   
Oracle               0.900753  0.827800  0.825512   0.836617  0.816054   

                          NLL     Brier  split  
approach                                        
DGE$_{20]$           0.507891  0.154712    2.0  
DGE$_{20}$ (concat)  0.502671  0.154198    2.0  
Naive                0.255922  0.065493    2.0  
Oracle               0.454684  0.124951    2.0  
                          AUC       Acc        F1  Precision    Recall  \
approach                                                                 
DGE$_{20]$           0.882827  0.806603  0.800894   0.825490  0.779701   
DGE$_{20}$ (concat)  0.883297  0.8